<a href="https://colab.research.google.com/github/analyticswithadam/Python/blob/main/GPT_4o_Supplier_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get the openai secret key
import pandas
import getpass
secret_key = getpass.getpass('Please enter your openai key: ')
from IPython.display import display, Markdown

In [ ]:
!pip install openai

In [ ]:
%pip install alpha_vantage

In [ ]:
import requests
import json
from IPython.display import display, Markdown

api_key = '6605TQULF0O9MKET'
ticker = 'DIS'  # Example ticker

# Function to fetch financial data
def fetch_financial_data(function, ticker, api_key):
    url = f"https://www.alphavantage.co/query?function={function}&symbol={ticker}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching {function} data.")
        return None

# Fetch income statement
income_statement = fetch_financial_data("INCOME_STATEMENT", ticker, api_key)



# Fetch balance sheet
balance_sheet = fetch_financial_data("BALANCE_SHEET", ticker, api_key)

In [ ]:
# Assuming you've already fetched the balance sheet data as 'balance_sheet'
quarterly_income_reports = income_statement['quarterlyReports'][:4]
quarterly_balance_reports = balance_sheet['quarterlyReports'][:4]

formatted_data = {"financialData": []}

# Iterate through the income reports and balance reports simultaneously
for income_report, balance_report in zip(quarterly_income_reports, quarterly_balance_reports):
    # Filter out entries with values "None" or an empty string for income statement
    formatted_income_report = {k: v for k, v in income_report.items() if v not in ["None", ""]}
    # Filter out entries with values "None" or an empty string for balance sheet
    formatted_balance_report = {k: v for k, v in balance_report.items() if v not in ["None", ""]}
    # Combine the data from both reports
    formatted_report = {**formatted_income_report, **formatted_balance_report}
    # Append the combined report to the financialData list
    formatted_data["financialData"].append(formatted_report)

# Convert the combined data to a JSON string
out_string = json.dumps(formatted_data, indent=4)
print(out_string)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=secret_key)

def get_response(prompt):
  # Create a request to the chat completions endpoint
  response = client.chat.completions.create(
    model="gpt-4o",
    # Assign the role and content for the message
    messages=
     [
        {
            "role": "system", "content": """

              Role
              -------
              You are a CPA Employed in Accounts Receivable in a Finance Function. Tasked with assessing supplier risk.

              Task
              ------
              Analyze the provided 4 quarters of financial data.
              Compare the financials between the four quarters, noting any significant movements or trends.
              Evaluate the implications of these movements for the company's financial health and operational efficiency.

              From the perspective of a company reciving services from this company highlight key risks in growing sales and getting paid on time strictly from
              what you have gathered from the financials.

              Decide whether the team need to investigate this supplier further for risk backed up the financials.

              Output
              ------
              Ensure the analysis reflects the specific relevant movements across the quarters,
              highlighting significant changes relevant to their viability as a major supplier.
              The output should strictly adhere to the data provided, avoiding assumptions or inferences not directly supported by the numbers.
              Im printing in markdown in python ensure summary tables readible and numeric text is legible
              Do not include any currency symbols, use 22.5M USD instead of $22.5M, display figures in millions and use thousand seperators where appropriate ie 23,100M


              Data
              -----

            """
        },
        {
            "role": "user", "content": prompt
        }
     ],
    temperature = 0.4,
    max_tokens=3000)
  return response.choices[0].message.content

In [36]:
output = get_response(out_string)

In [37]:
display(Markdown(output))

# Financial Analysis of Supplier

## Quarterly Financial Data Summary

Below is a summary of the key financial metrics for the supplier over the last four quarters:

| Metric                          | 2024-03-31 | 2023-12-31 | 2023-09-30 | 2023-06-30 |
|---------------------------------|------------|------------|------------|------------|
| **Total Revenue (M)**           | 22,083     | 23,549     | 21,241     | 22,330     |
| **Gross Profit (M)**            | 15,910     | 19,415     | 16,069     | 15,461     |
| **Operating Income (M)**        | 3,845      | 3,876      | 2,976      | 3,559      |
| **Net Income (M)**              | -20        | 1,911      | 264        | -460       |
| **Total Assets (M)**            | 195,110    | 197,774    | 205,579    | 203,783    |
| **Total Liabilities (M)**       | 95,858     | 97,053     | 106,302    | 106,173    |
| **Total Shareholder Equity (M)**| 99,252     | 100,721    | 99,277     | 97,610     |
| **Cash and Cash Equivalents (M)**| 6,635      | 7,192      | 14,182     | 11,458     |
| **Current Net Receivables (M)** | 12,026     | 14,115     | 12,330     | 13,112     |
| **Current Liabilities (M)**     | 32,874     | 31,033     | 31,139     | 28,234     |
| **Current Debt (M)**            | 6,789      | 6,087      | 4,367      | 2,645      |

## Analysis of Financial Movements

### Revenue and Profitability
- **Total Revenue**: There was a noticeable decline from 23,549M in Q4 2023 to 22,083M in Q1 2024. This represents a drop of approximately 6.2%.
- **Gross Profit**: Gross profit has decreased significantly from 19,415M in Q4 2023 to 15,910M in Q1 2024, a reduction of about 18%.
- **Operating Income**: Operating income remained relatively stable between Q4 2023 and Q1 2024, with a slight decrease from 3,876M to 3,845M.
- **Net Income**: Net income fluctuated significantly, with a notable drop from 1,911M in Q4 2023 to a loss of 20M in Q1 2024. This indicates potential issues in managing costs or other non-operating expenses.

### Asset and Liability Management
- **Total Assets**: Total assets decreased from 205,579M in Q3 2023 to 195,110M in Q1 2024, indicating a reduction in the asset base.
- **Total Liabilities**: Total liabilities decreased from 106,302M in Q3 2023 to 95,858M in Q1 2024, which is a positive sign of reducing debt or other liabilities.
- **Total Shareholder Equity**: Shareholder equity has remained relatively stable, with a slight decrease from 100,721M in Q4 2023 to 99,252M in Q1 2024.

### Liquidity and Solvency
- **Cash and Cash Equivalents**: There was a significant reduction in cash and cash equivalents from 14,182M in Q3 2023 to 6,635M in Q1 2024, which could impact the company's liquidity.
- **Current Net Receivables**: Current net receivables decreased from 14,115M in Q4 2023 to 12,026M in Q1 2024, indicating potential collection issues or reduced sales.
- **Current Liabilities**: Current liabilities increased from 28,234M in Q2 2023 to 32,874M in Q1 2024, which could strain the company's short-term financial health.
- **Current Debt**: Current debt increased from 2,645M in Q2 2023 to 6,789M in Q1 2024, indicating higher short-term borrowing.

## Key Risks and Recommendations

### Risks
1. **Revenue Decline**: The decline in total revenue and gross profit indicates potential challenges in maintaining sales growth or managing cost of goods sold.
2. **Net Income Volatility**: The significant fluctuations in net income, including a loss in Q1 2024, raise concerns about the company's profitability and cost management.
3. **Liquidity Concerns**: The substantial reduction in cash and cash equivalents could impact the company's ability to meet short-term obligations and invest in growth opportunities.
4. **Increased Current Liabilities and Debt**: The increase in current liabilities and short-term debt could strain the company's liquidity and operational efficiency.

### Recommendations
- **Further Investigation**: Given the significant movements in revenue, profitability, and liquidity metrics, it is recommended to investigate this supplier further to assess the underlying causes and potential risks.
- **Monitor Receivables**: Closely monitor the supplier's receivables and payment patterns to ensure timely payments and reduce the risk of bad debts.
- **Evaluate Financial Stability**: Assess the supplier's financial stability and ability to meet short-term obligations, considering the increase in current liabilities and debt.

Based on the analysis, further investigation into the supplier's financial health and operational efficiency is warranted to mitigate potential risks in growing sales and ensuring timely payments.